In [54]:
import pandas as pd

def preprocess_sales_data_flexible(df, n_past_week=6, n_future_week=1, start_date=None, end_date=None):
    """
    Preprocesses the sales dataset to generate time-based features and future sales aggregation.

    Parameters:
    df (pd.DataFrame): The sales data
    n_past_week (int): Number of past weeks to use for features (default=6)
    n_future_week (int): Number of future weeks to predict aggregated sales (default=1)
    start_date (str): Optional start date for filtering data (format: 'YYYY-MM')
    end_date (str): Optional end date for filtering data (format: 'YYYY-MM')

    Returns:
    pd.DataFrame: Processed dataset with time-series features and future target sales
    """
    # Convert Transaction Date to datetime format
    df["Transaction Date"] = pd.to_datetime(df["Transaction Date"])

    # Extract Week (starting on Sunday) for aggregation
    df["Year_Week"] = df["Transaction Date"].dt.to_period("W-SUN").apply(lambda x: x.start_time.strftime('%Y-%m-%d'))

    # Convert Discount Applied to binary values (True=1, False=0)
    df["Discount Applied"] = df["Discount Applied"].fillna(False).astype(int)

    # Aggregate basic features
    grouped = df.groupby("Year_Week").agg(
        weekly_avg_repeat_purchases=("Customer ID", lambda x: x.duplicated().mean()),
        total_sales=("Total Spent", "sum"),
        unique_transactions=("Transaction ID", "nunique"),
        unique_customers=("Customer ID", "nunique"),
        discounted_transactions=("Discount Applied", "sum"),
    ).reset_index()

    # Aggregate category sales but shift to keep only t-1 to t-n values
    category_sales = df.groupby(["Year_Week", "Category"])["Total Spent"].sum().unstack(fill_value=0)
    # Shift to remove current week's data

    # Aggregate unique transactions per payment method
    payment_transactions = df.groupby(["Year_Week", "Payment Method"])["Transaction ID"].nunique().unstack(fill_value=0)

    # Aggregate unique transactions per location
    location_transactions = df.groupby(["Year_Week", "Location"])["Transaction ID"].nunique().unstack(fill_value=0)

    # Merge all aggregated data
    final_df = grouped.merge(category_sales, on="Year_Week", how="left") \
        .merge(payment_transactions, on="Year_Week", how="left") \
        .merge(location_transactions, on="Year_Week", how="left")

    # Fill NaNs resulting from merging
    # Allow NaN values for rows without data
    final_df.fillna(float('nan'), inplace=True)

    # Generate past weeks features (t-1 to t-N)
    feature_cols = [col for col in final_df.columns if col != "Year_Week"]
    past_week_cols = ["unique_transactions", "unique_customers", "discounted_transactions"] + feature_cols
    for col in feature_cols:
        for i in range(1, n_past_week + 1):
            final_df[f"{col}_t-{i}"] = final_df[col].shift(i)

    # Generate future target sales (T+1 to T+n_future_week)
    for i in range(1, n_future_week + 1):
        final_df[f"y_{i}"] = (
            final_df["total_sales"].shift(-i)
        )

    # Apply date filtering if specified
    if start_date:
        final_df = final_df[final_df["Year_Week"] >= start_date]
    if end_date:
        final_df = final_df[final_df["Year_Week"] <= end_date]

    return final_df


In [55]:
data = pd.read_csv('data/retail_store_sales.csv')

In [56]:
preprocess_sales_data_flexible(data,5,4)

,Year_Week,weekly_avg_repeat_purchases,total_sales,unique_transactions,unique_customers,discounted_transactions,Beverages,Butchers,Computers and electric accessories,Electric household essentials,...,In-store_t-5,Online_t-1,Online_t-2,Online_t-3,Online_t-4,Online_t-5,y_1,y_2,y_3,y_4
0,2021-12-27,0.263158,2778.0,19,14,8,461.0,604.5,150.0,350.0,...,NaN,NaN,NaN,NaN,NaN,NaN,9666.5,12803.0,13441.0,11912.5
1,2022-01-03,0.714286,9666.5,84,24,26,747.0,1239.5,1823.0,1028.0,...,NaN,6.0,NaN,NaN,NaN,NaN,12803.0,13441.0,11912.5,10810.0
2,2022-01-10,0.747368,12803.0,95,24,39,1718.5,2277.5,1657.5,1682.0,...,NaN,36.0,6.0,NaN,NaN,NaN,13441.0,11912.5,10810.0,11102.5
3,2022-01-17,0.744898,13441.0,98,25,38,1739.0,1789.0,1442.0,1637.5,...,NaN,53.0,36.0,6.0,NaN,NaN,11912.5,10810.0,11102.5,12539.0
4,2022-01-24,0.687500,11912.5,80,25,31,471.5,2127.5,2193.0,601.0,...,NaN,45.0,53.0,36.0,6.0,NaN,10810.0,11102.5,12539.0,8757.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2024-12-16,0.689189,9124.5,74,23,21,457.0,467.0,816.5,1611.5,...,35.0,41.0,36.0,47.0,43.0,45.0,12620.5,11858.0,9902.5,6904.0
156,2024-12-23,0.728261,12620.5,92,25,34,2084.0,576.0,2589.5,1326.5,...,33.0,36.0,41.0,36.0,47.0,43.0,11858.0,9902.5,6904.0,NaN
157,2024-12-30,0.744898,11858.0,98,25,25,1887.0,1819.0,1708.5,1332.0,...,34.0,48.0,36.0,41.0,36.0,47.0,9902.5,6904.0,NaN,NaN
158,2025-01-06,0.679487,9902.5,78,25,24,467.0,1790.0,1551.0,943.0,...,43.0,42.0,48.0,36.0,41.0,36.0,6904.0,NaN,NaN,NaN
